In [1]:
#import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

# set default figure size
plt.rcParams['figure.figsize'] = (4, 4) 

# Aviation Accident Capstone:
## *Part II: Pre-Processing & Dummy Variables*
Created by: Katy Christensen <br>
Created on: September 26, 2022 <br>
Created for: BrainStation Data Science Bootcamp Capstone<br>
Notebook 2 of 5<br>

Previous Notebook: *Part I: Cleaning Data* <br>
Upcoming Notebook: *Part III: Logistic Regresssion Model*

--------------
### Overview
This Jupyter Notebook continues the pre-processing and deals with the categorical features. Models rely on numerical data and so categorical must be processed into ordinal or binary values. Ordinal provides a weight based on importance (i.e. a scale from 1-10) and binary provides an even weight across the categorical values. All of the categorical values were binarized in this notebook. 

**Binary Categorical Feautre Methodology:** If a feature values were yes or no, the yes values were imputed with '1' and no with '0' with NumPys '.where' functionality. 

**Multi-Category Feature Methodology:** For features with multiple features (i.e. 'NONE', 'IFR', 'VFR', etc.), a process called *OneHoteEncoding* is used. This takes each of the categorical values and turns them into an additional feature. Within that feature, a 1 represents if that category is true. This process is replicated across the baord. Following the OneHotEncoding, one of the features related to the original feature must be dropped in order to avoid collinearity. The 'new' features can then be concatenated back on to the main dataframe. 

*Note: Any missed cleaning or features that required a final decision from* Part I *were also dealt with in this notebook.*

## Table of Contents
[1. Data Summary](#Step-1) <br>
- [Data Shape](#data-shape) <br>
- [Check for Nulls](#nulls) <br>
- [Duplicates Check](#duplicates) <br>
- [Data Type Review](#review-dtypes) <br>

[2. Dummy Variables](#Step-2) <br>
- [Dependent Variable: Accident Fatality](#fatal)
- [Aircraft Information Features](#acft-info) <br>
- [Departure Information Features](#dprt-info)<br>
- [Destination Features](#Numeric-Columns)
- [Event Information Features](#ev-info)<br>
- [Weather Information Features](#wx-inf0)<br>
- [Crew Information Features](#Numeric-Columns)

[3. Verify Data](#Step-3) <br>
[4. Results & Summary](#Step-4) <br>

<a id='Step-1'></a>
## 1. Data Summary

In [2]:
#Load data
ntsb08 = pd.read_csv('data/ntsb08_clean.csv')
ntsb08.head()

,Unnamed: 0,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,...,crew_age,crew_sex,crew_res_state,med_certf,med_crtf_vldty,seatbelts_used,crew_tox_perf,seat_occ_pic,total_hours,durt_ev_med
0,0,N,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,...,42.0,M,AK,CL2,VWW,Y,N,LEFT,5350.0,286.0
1,1,N,121,IFR,N,MINR,GRD,NONE,Boeing,747-100,...,53.0,M,AZ,CL1,VWW,Y,N,LEFT,23000.0,15.0
2,2,N,135,CVFR,N,SUBS,NONE,NONE,Piper,PA-31-350,...,39.0,M,AK,CL1,VNOW,Y,N,LEFT,10000.0,35.0
3,3,N,091,NONE,N,SUBS,NONE,NONE,Cessna,172,...,25.0,F,AK,CL3,VNOW,Y,N,LEFT,450.0,438.0
4,4,N,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,...,38.0,M,AK,CL2,VNOW,Y,N,LEFT,854.0,148.0


When the previous CSV file was exported, an index was included and created an extra column on import into this Notebook. The purpose of this project is to predict the fatality of an aviation accident, so the additional index, `ev_id`, and `Aircraft_Key` columns will be dropped. `ev_id` is the unique identifier for an accident and `Aircraft_Key` is the identifier if there are more than one aircraft involved in the accident. 

<a id='data-shape'></a>
### What is the shape of the data? 
The data set containes 64,192 rows of data and 64 columns (or features/variables) including an 'Unnamed: 0' column. This is an index column that was included when the file was exported from the last Notebook. 

In [3]:
#Check shape
ntsb08.shape

(64192, 64)

In [4]:
ntsb08 = ntsb08.drop(['Unnamed: 0'], axis=1)
#Verify columns were dropped
ntsb08.head()

,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,...,crew_age,crew_sex,crew_res_state,med_certf,med_crtf_vldty,seatbelts_used,crew_tox_perf,seat_occ_pic,total_hours,durt_ev_med
0,N,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,...,42.0,M,AK,CL2,VWW,Y,N,LEFT,5350.0,286.0
1,N,121,IFR,N,MINR,GRD,NONE,Boeing,747-100,750000.0,...,53.0,M,AZ,CL1,VWW,Y,N,LEFT,23000.0,15.0
2,N,135,CVFR,N,SUBS,NONE,NONE,Piper,PA-31-350,7369.0,...,39.0,M,AK,CL1,VNOW,Y,N,LEFT,10000.0,35.0
3,N,091,NONE,N,SUBS,NONE,NONE,Cessna,172,2300.0,...,25.0,F,AK,CL3,VNOW,Y,N,LEFT,450.0,438.0
4,N,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,...,38.0,M,AK,CL2,VNOW,Y,N,LEFT,854.0,148.0


In [5]:
#Check shape
ntsb08.shape

(64192, 63)

<a id='nulls'></a>
### Null Check
Even though this data was cleaned in the previous Jupyter Notebook, check for any nulls - there are no nulls

In [6]:
ntsb08.isna().sum().sum()

0

<a id='duplicates'></a>
### Duplicates Check
There are 12 duplicated rows; these need to be dropped.

In [7]:
ntsb08.duplicated().sum()

12

In [8]:
#drop duplicates
ntsb08 = ntsb08.drop_duplicates()

<a id='review-dtype'></a>
### Review the data types & convert as necessary

In [9]:
ntsb08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64180 entries, 0 to 64191
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   acft_missing             64180 non-null  object 
 1   far_part                 64180 non-null  object 
 2   flt_plan_filed           64180 non-null  object 
 3   flight_plan_activated    64180 non-null  object 
 4   damage                   64180 non-null  object 
 5   acft_fire                64180 non-null  object 
 6   acft_expl                64180 non-null  object 
 7   acft_make                64180 non-null  object 
 8   acft_model               64180 non-null  object 
 9   cert_max_gr_wt           64180 non-null  float64
 10  acft_category            64180 non-null  object 
 11  homebuilt                64180 non-null  object 
 12  total_seats              64180 non-null  float64
 13  num_eng                  64180 non-null  float64
 14  elt_install           

Overall there is a blend of float, integer, and object columns. Some of the float columns can be converted to integers, but since this is not a large dataset is not strictly necessary. `far_part1` should reamin as an object and not converted because the numbers are representative of a type of aircraft operation. 

In [10]:
#Review the categorical columns
print('Text Columns:\n') 
cat_list = list(ntsb08.select_dtypes(exclude='number'))
print('Total Categorical Features:', len(cat_list))
print(cat_list)

Text Columns:

Total Categorical Features: 44
['acft_missing', 'far_part', 'flt_plan_filed', 'flight_plan_activated', 'damage', 'acft_fire', 'acft_expl', 'acft_make', 'acft_model', 'acft_category', 'homebuilt', 'elt_install', 'elt_oper', 'elt_aided_loc_ev', 'certs_held', 'type_fly', 'second_pilot', 'dprt_state', 'dprt_country', 'dest_state', 'dest_country', 'site_seeing', 'air_medical', 'ev_type', 'ev_date', 'ev_dow', 'ev_tmzn', 'ev_state', 'mid_air', 'on_ground_collision', 'ev_nr_apt_loc', 'light_cond', 'sky_cond_nonceil', 'sky_cond_ceil', 'ev_highest_injury', 'wx_cond_basic', 'crew_category', 'crew_sex', 'crew_res_state', 'med_certf', 'med_crtf_vldty', 'seatbelts_used', 'crew_tox_perf', 'seat_occ_pic']


In [11]:
#Review the numeric columns
print('Numeric Columns:\n')
num_list = list(ntsb08.select_dtypes(include='number'))
print('Total Numeric Features:', len(num_list))
print(num_list)

Numeric Columns:

Total Numeric Features: 19
['cert_max_gr_wt', 'total_seats', 'num_eng', 'phase_flt_spec', 'commercial_space_flight', 'unmanned', 'ifr_equipped_cert', 'elt_mounted_aircraft', 'elt_connected_antenna', 'ev_time', 'ev_year', 'ev_month', 'vis_sm', 'wx_temp', 'wind_dir_deg', 'wind_vel_kts', 'crew_age', 'total_hours', 'durt_ev_med']


---
<a id='Step-2'></a>
### 2. Create Dummy Variables
---
The strategy for creating dummy variables is to convert them into numerical data. Most columns will be binarized into a yes (1)/ no (0) format. For example, when an aircraft was missing (`acft_missing`) then yes or 'Y' values will be converted to 1 otherwise it will be converted to a 0. For features going beyond a simple yes/no, then each unqiue value will be converted to a new column and then binarized.  

There are 44 categorical features which will be converted (see cell below).

In [12]:
#Review the categorical columns
print('Text Columns:\n') 
cat_list = list(ntsb08.select_dtypes(exclude='number'))
print(cat_list, '\n', f'Categorical Features: {len(cat_list)}')

Text Columns:

['acft_missing', 'far_part', 'flt_plan_filed', 'flight_plan_activated', 'damage', 'acft_fire', 'acft_expl', 'acft_make', 'acft_model', 'acft_category', 'homebuilt', 'elt_install', 'elt_oper', 'elt_aided_loc_ev', 'certs_held', 'type_fly', 'second_pilot', 'dprt_state', 'dprt_country', 'dest_state', 'dest_country', 'site_seeing', 'air_medical', 'ev_type', 'ev_date', 'ev_dow', 'ev_tmzn', 'ev_state', 'mid_air', 'on_ground_collision', 'ev_nr_apt_loc', 'light_cond', 'sky_cond_nonceil', 'sky_cond_ceil', 'ev_highest_injury', 'wx_cond_basic', 'crew_category', 'crew_sex', 'crew_res_state', 'med_certf', 'med_crtf_vldty', 'seatbelts_used', 'crew_tox_perf', 'seat_occ_pic'] 
 Categorical Features: 44


<a id='fatal'></a>
### Dependent Variable: 
#### Fatal Accident | `ev_highest_injury`

In [13]:
ntsb08['ev_highest_injury'] = ntsb08['ev_highest_injury'].str.upper()
ntsb08['ev_highest_injury'].value_counts()

NONE    35197
FATL    12068
MINR     9541
SERS     7372
UNKN        2
Name: ev_highest_injury, dtype: int64

In [14]:
#Binarize for fatal (1) or not fatal (0)
ntsb08['ev_highest_injury'] = np.where(ntsb08['ev_highest_injury'] == 'FATL', 1, 0)
#Verify
ntsb08['ev_highest_injury'].value_counts()

0    52112
1    12068
Name: ev_highest_injury, dtype: int64

<a id='acft-info'></a>
### Aircraft Information Categorical Features
#####  Aircraft Missing | `acft_missing`

In [15]:
#Find count by category value
ntsb08['acft_missing'].value_counts()

N    63975
Y      205
Name: acft_missing, dtype: int64

In [16]:
#Binarize categories with 1 for 'Y'
ntsb08['acft_missing'] = np.where(ntsb08['acft_missing'] == 'Y', 1, 0)
ntsb08['acft_missing']

0        0
1        0
2        0
3        0
4        0
        ..
64187    0
64188    0
64189    0
64190    0
64191    0
Name: acft_missing, Length: 64180, dtype: int32

In [17]:
#verify counts are the same
ntsb08['acft_missing'].value_counts()

0    63975
1      205
Name: acft_missing, dtype: int64

##### Federal Aviation Regulation (FAR) Part | `far_part`

In [18]:
ntsb08['far_part'].value_counts()

091     53991
137      3748
135      3111
121      2167
133       349
PUBU      332
103       191
129       189
091F       36
125        28
ARMF       15
NUSC       13
NUSN        8
091K        2
Name: far_part, dtype: int64

In [19]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['far_part'])

# Fit the OneHotEncoder to the subcategory column and transform
# It expects a 2D array, so we first convert the column into a DataFrame
encoded_farpt = ohe.fit_transform(subcategory)
encoded_farpt

<64180x14 sparse matrix of type '<class 'numpy.float64'>'
	with 64180 stored elements in Compressed Sparse Row format>

In [20]:
# Convert from sparse matrix to dense
dense_array = encoded_farpt.toarray()
dense_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
#check category 'names'
ohe.categories_

[array(['091', '091F', '091K', '103 ', '121 ', '125 ', '129 ', '133 ',
        '135 ', '137 ', 'ARMF', 'NUSC', 'NUSN', 'PUBU'], dtype=object)]

In [22]:
# Put into a dataframe to get column names
encoded_farpt = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_farpt = encoded_farpt.add_prefix('farpt_')

# Add original back in (just to check)
encoded_farpt['far_part'] = ntsb08['far_part']

# Show
encoded_farpt.head()

,farpt_091,farpt_091F,farpt_091K,farpt_103,farpt_121,farpt_125,farpt_129,farpt_133,farpt_135,farpt_137,farpt_ARMF,farpt_NUSC,farpt_NUSN,farpt_PUBU,far_part
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,135
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,121
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,135
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,091
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,135


In [23]:
#Drop the final column
encoded_farpt = encoded_farpt.drop(['far_part'], axis=1)

#check dataframe
encoded_farpt.head(1)

,farpt_091,farpt_091F,farpt_091K,farpt_103,farpt_121,farpt_125,farpt_129,farpt_133,farpt_135,farpt_137,farpt_ARMF,farpt_NUSC,farpt_NUSN,farpt_PUBU
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [24]:
#View percentage across columns
encoded_farpt.sum().sort_values(ascending=False)/encoded_farpt.shape[0]*100

farpt_091     84.124338
farpt_137      5.839825
farpt_135      4.847304
farpt_121      3.376441
farpt_133      0.543783
farpt_PUBU     0.517295
farpt_103      0.297600
farpt_129      0.294484
farpt_091F     0.056092
farpt_125      0.043627
farpt_ARMF     0.023372
farpt_NUSC     0.020256
farpt_NUSN     0.012465
farpt_091K     0.003116
dtype: float64

In [25]:
#Drop 1x column to avoid collinearity
encoded_farpt = encoded_farpt.drop(['farpt_091K'], axis=1)
#Check dataframe
encoded_farpt.head(1)

,farpt_091,farpt_091F,farpt_103,farpt_121,farpt_125,farpt_129,farpt_133,farpt_135,farpt_137,farpt_ARMF,farpt_NUSC,farpt_NUSN,farpt_PUBU
0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [26]:
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_farpt], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,...,"(farpt_121 ,)","(farpt_125 ,)","(farpt_129 ,)","(farpt_133 ,)","(farpt_135 ,)","(farpt_137 ,)","(farpt_ARMF,)","(farpt_NUSC,)","(farpt_NUSN,)","(farpt_PUBU,)"
0,0.0,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,121,IFR,N,MINR,GRD,NONE,Boeing,747-100,750000.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,135,CVFR,N,SUBS,NONE,NONE,Piper,PA-31-350,7369.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,091,NONE,N,SUBS,NONE,NONE,Cessna,172,2300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,135,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
#Drop initial column
ntsb08 = ntsb08.drop(['far_part'], axis=1)
#Check dataframe
ntsb08.head()

,acft_missing,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,acft_category,...,"(farpt_121 ,)","(farpt_125 ,)","(farpt_129 ,)","(farpt_133 ,)","(farpt_135 ,)","(farpt_137 ,)","(farpt_ARMF,)","(farpt_NUSC,)","(farpt_NUSN,)","(farpt_PUBU,)"
0,0.0,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,IFR,N,MINR,GRD,NONE,Boeing,747-100,750000.0,AIR,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,CVFR,N,SUBS,NONE,NONE,Piper,PA-31-350,7369.0,AIR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,NONE,N,SUBS,NONE,NONE,Cessna,172,2300.0,AIR,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


###### Type of Filed Flight Plan | `flt_plan_filed`

In [28]:
ntsb08['flt_plan_filed'].value_counts()

NONE    48915
IFR      8255
VFR      4679
CVFR     2160
VFIF      126
MVFR       45
Name: flt_plan_filed, dtype: int64

In [29]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['flt_plan_filed'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_fltpln = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_fltpln.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_fltpln = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_fltpln = encoded_fltpln.add_prefix('fltpln_')

# Add original back in (just to check)
encoded_fltpln['flt_plan_filed'] = ntsb08['flt_plan_filed']

# Show
encoded_fltpln.head()

[array(['CVFR', 'IFR ', 'MVFR', 'NONE', 'VFIF', 'VFR ', nan], dtype=object)]


,fltpln_CVFR,fltpln_IFR,fltpln_MVFR,fltpln_NONE,fltpln_VFIF,fltpln_VFR,fltpln_nan,flt_plan_filed
0,1,0,0,0,0,0,0,CVFR
1,0,1,0,0,0,0,0,IFR
2,1,0,0,0,0,0,0,CVFR
3,0,0,0,1,0,0,0,NONE
4,1,0,0,0,0,0,0,CVFR


In [30]:
#Drop the final column
encoded_fltpln = encoded_fltpln.drop(['flt_plan_filed'], axis=1)

#View percentage across columns
encoded_fltpln.sum().sort_values(ascending=False)

fltpln_NONE    48915
fltpln_IFR      8255
fltpln_VFR      4679
fltpln_CVFR     2160
fltpln_VFIF      126
fltpln_MVFR       45
fltpln_nan        12
dtype: int64

In [31]:
#Drop 1x column to avoid collinearity
encoded_fltpln = encoded_fltpln.drop(['fltpln_MVFR'], axis=1)

In [32]:
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_fltpln], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,acft_category,...,"(farpt_ARMF,)","(farpt_NUSC,)","(farpt_NUSN,)","(farpt_PUBU,)","(fltpln_CVFR,)","(fltpln_IFR ,)","(fltpln_NONE,)","(fltpln_VFIF,)","(fltpln_VFR ,)","(fltpln_nan,)"
0,0.0,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
1,0.0,IFR,N,MINR,GRD,NONE,Boeing,747-100,750000.0,AIR,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,0.0,CVFR,N,SUBS,NONE,NONE,Piper,PA-31-350,7369.0,AIR,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
3,0.0,NONE,N,SUBS,NONE,NONE,Cessna,172,2300.0,AIR,...,0.0,0.0,0.0,0.0,0,0,1,0,0,0
4,0.0,CVFR,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0


In [33]:
#Drop original column
ntsb08 = ntsb08.drop(['flt_plan_filed'], axis=1)
ntsb08.head()

,acft_missing,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,acft_category,homebuilt,...,"(farpt_ARMF,)","(farpt_NUSC,)","(farpt_NUSN,)","(farpt_PUBU,)","(fltpln_CVFR,)","(fltpln_IFR ,)","(fltpln_NONE,)","(fltpln_VFIF,)","(fltpln_VFR ,)","(fltpln_nan,)"
0,0.0,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,N,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
1,0.0,N,MINR,GRD,NONE,Boeing,747-100,750000.0,AIR,N,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,0.0,N,SUBS,NONE,NONE,Piper,PA-31-350,7369.0,AIR,N,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0
3,0.0,N,SUBS,NONE,NONE,Cessna,172,2300.0,AIR,N,...,0.0,0.0,0.0,0.0,0,0,1,0,0,0
4,0.0,N,SUBS,NONE,NONE,Cessna,207,3800.0,AIR,N,...,0.0,0.0,0.0,0.0,1,0,0,0,0,0


##### Flight Plan Activated | `flight_plan_activated`

In [34]:
ntsb08['flight_plan_activated'].value_counts().sort_values(ascending=False)

N    61000
Y     3073
U      107
Name: flight_plan_activated, dtype: int64

In [35]:
ntsb08['flight_plan_activated'] = np.where(ntsb08['flight_plan_activated'] == 'Y', 1, 0)
ntsb08['flight_plan_activated'].value_counts().sort_values(ascending=False)

0    61119
1     3073
Name: flight_plan_activated, dtype: int64

##### Damage | `damage`

In [36]:
ntsb08['damage'].value_counts().sort_values(ascending=False)

SUBS    45287
DEST    15302
MINR     1898
NONE     1684
UNK         9
Name: damage, dtype: int64

**Comment:** Unknowns will be imputed with SUBS, which is the leading category

In [37]:
#Replace UNK with SUBS
ntsb08['damage'] = np.where(ntsb08['damage'] == 'UNK', 'SUBS', ntsb08['damage'])
#Replace UNK with SUBS
ntsb08['damage'] = np.where(ntsb08['damage'] == 'UNK ', 'SUBS', ntsb08['damage'])

In [38]:
ntsb08['damage'].value_counts().sort_values(ascending=False)

SUBS    45296
DEST    15302
MINR     1898
NONE     1684
Name: damage, dtype: int64

In [39]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['damage'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_dmg = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_dmg.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_dmg = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_dmg = encoded_dmg.add_prefix('dmg_')

# Add original back in (just to check)
encoded_dmg['damage'] = ntsb08['damage']

# Show
encoded_dmg.head()

[array(['DEST', 'MINR', 'NONE', 'SUBS', nan], dtype=object)]


,dmg_DEST,dmg_MINR,dmg_NONE,dmg_SUBS,dmg_nan,damage
0,0,0,0,1,0,SUBS
1,0,1,0,0,0,MINR
2,0,0,0,1,0,SUBS
3,0,0,0,1,0,SUBS
4,0,0,0,1,0,SUBS


In [40]:
#Drop the final column
encoded_dmg = encoded_dmg.drop(['damage'], axis=1)

#View percentage across columns
encoded_dmg.sum().sort_values(ascending=False)

dmg_SUBS    45296
dmg_DEST    15302
dmg_MINR     1898
dmg_NONE     1684
dmg_nan        12
dtype: int64

In [41]:
#Drop 1x column to avoid collinearity
encoded_dmg = encoded_dmg.drop(['dmg_NONE'], axis=1)

In [42]:
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_dmg], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['damage'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,acft_fire,acft_expl,acft_make,acft_model,cert_max_gr_wt,acft_category,homebuilt,total_seats,...,"(fltpln_CVFR,)","(fltpln_IFR ,)","(fltpln_NONE,)","(fltpln_VFIF,)","(fltpln_VFR ,)","(fltpln_nan,)","(dmg_DEST,)","(dmg_MINR,)","(dmg_SUBS,)","(dmg_nan,)"
0,0.0,0,NONE,NONE,Cessna,207,3800.0,AIR,N,2.0,...,1,0,0,0,0,0,0,0,1,0
1,0.0,0,GRD,NONE,Boeing,747-100,750000.0,AIR,N,9.0,...,0,1,0,0,0,0,0,1,0,0
2,0.0,0,NONE,NONE,Piper,PA-31-350,7369.0,AIR,N,6.0,...,1,0,0,0,0,0,0,0,1,0
3,0.0,0,NONE,NONE,Cessna,172,2300.0,AIR,N,4.0,...,0,0,1,0,0,0,0,0,1,0
4,0.0,0,NONE,NONE,Cessna,207,3800.0,AIR,N,6.0,...,1,0,0,0,0,0,0,0,1,0


##### Aircraft Fire | `acft_fire`

In [43]:
ntsb08['acft_fire'].value_counts().sort_values(ascending=False)

NONE    56651
GRD      6101
IFLT      576
UNK       468
UNK       200
BOTH      172
UORG       12
Name: acft_fire, dtype: int64

**Comment:** Unknown will be rolled into the 'NONE' category. 

In [44]:
#Replace UNK with SUBS
ntsb08['acft_fire'] = np.where(ntsb08['acft_fire'] == 'UNK', 'SUBS', ntsb08['acft_fire'])
#Replace UNK with SUBS
ntsb08['acft_fire'] = np.where(ntsb08['acft_fire'] == 'UNK ', 'SUBS', ntsb08['acft_fire'])
#Verify
ntsb08['acft_fire'].value_counts().sort_values(ascending=False)

NONE    56651
GRD      6101
SUBS      668
IFLT      576
BOTH      172
UORG       12
Name: acft_fire, dtype: int64

In [45]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['acft_fire'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_fire = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_fire.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_fire = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_fire = encoded_fire.add_prefix('fire_')

# Add original back in (just to check)
encoded_fire['acft_fire'] = ntsb08['acft_fire']

# Show
encoded_fire.head()

[array(['BOTH', 'GRD ', 'IFLT', 'NONE', 'SUBS', 'UORG', nan], dtype=object)]


,fire_BOTH,fire_GRD,fire_IFLT,fire_NONE,fire_SUBS,fire_UORG,fire_nan,acft_fire
0,0,0,0,1,0,0,0,NONE
1,0,1,0,0,0,0,0,GRD
2,0,0,0,1,0,0,0,NONE
3,0,0,0,1,0,0,0,NONE
4,0,0,0,1,0,0,0,NONE


In [46]:
#Drop the final column
encoded_fire = encoded_fire.drop(['acft_fire'], axis=1)

#View percentage across columns
encoded_fire.sum().sort_values(ascending=False)

fire_NONE    56651
fire_GRD      6101
fire_SUBS      668
fire_IFLT      576
fire_BOTH      172
fire_UORG       12
fire_nan        12
dtype: int64

In [47]:
#Drop 1x column to avoid collinearity
encoded_fire = encoded_fire.drop(['fire_UORG'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_fire], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['acft_fire'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,acft_expl,acft_make,acft_model,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,...,"(dmg_DEST,)","(dmg_MINR,)","(dmg_SUBS,)","(dmg_nan,)","(fire_BOTH,)","(fire_GRD ,)","(fire_IFLT,)","(fire_NONE,)","(fire_SUBS,)","(fire_nan,)"
0,0.0,0,NONE,Cessna,207,3800.0,AIR,N,2.0,1.0,...,0,0,1,0,0,0,0,1,0,0
1,0.0,0,NONE,Boeing,747-100,750000.0,AIR,N,9.0,4.0,...,0,1,0,0,0,1,0,0,0,0
2,0.0,0,NONE,Piper,PA-31-350,7369.0,AIR,N,6.0,2.0,...,0,0,1,0,0,0,0,1,0,0
3,0.0,0,NONE,Cessna,172,2300.0,AIR,N,4.0,1.0,...,0,0,1,0,0,0,0,1,0,0
4,0.0,0,NONE,Cessna,207,3800.0,AIR,N,6.0,1.0,...,0,0,1,0,0,0,0,1,0,0


##### Aircraft Explosion | `acft_expl`

In [48]:
ntsb08['acft_expl'].value_counts().sort_values(ascending=False)

NONE    59958
GRD      1761
UNK      1686
UNK       546
IFLT      166
BOTH       50
UORG       13
Name: acft_expl, dtype: int64

**Comment:** Unknown will be rolled into the 'NONE' category. 

In [49]:
#Replace UNK with SUBS
ntsb08['acft_expl'] = np.where(ntsb08['acft_expl'] == 'UNK', 'SUBS', ntsb08['acft_expl'])
#Replace UNK with SUBS
ntsb08['acft_expl'] = np.where(ntsb08['acft_expl'] == 'UNK ', 'SUBS', ntsb08['acft_expl'])
#Verify
ntsb08['acft_expl'].value_counts().sort_values(ascending=False)

NONE    59958
SUBS     2232
GRD      1761
IFLT      166
BOTH       50
UORG       13
Name: acft_expl, dtype: int64

In [50]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['acft_expl'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_expl = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_expl.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_expl = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_expl = encoded_expl.add_prefix('expl_')

# Add original back in (just to check)
encoded_expl['acft_expl'] = ntsb08['acft_expl']

# Show
encoded_expl.head()

[array(['BOTH', 'GRD ', 'IFLT', 'NONE', 'SUBS', 'UORG', nan], dtype=object)]


,expl_BOTH,expl_GRD,expl_IFLT,expl_NONE,expl_SUBS,expl_UORG,expl_nan,acft_expl
0,0,0,0,1,0,0,0,NONE
1,0,0,0,1,0,0,0,NONE
2,0,0,0,1,0,0,0,NONE
3,0,0,0,1,0,0,0,NONE
4,0,0,0,1,0,0,0,NONE


In [51]:
#Drop the final column
encoded_expl = encoded_expl.drop(['acft_expl'], axis=1)

#View percentage across columns
encoded_expl.sum().sort_values(ascending=False)

expl_NONE    59958
expl_SUBS     2232
expl_GRD      1761
expl_IFLT      166
expl_BOTH       50
expl_UORG       13
expl_nan        12
dtype: int64

In [52]:
#Drop 1x column to avoid collinearity
encoded_expl = encoded_expl.drop(['expl_UORG'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_expl], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['acft_expl'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,acft_make,acft_model,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,elt_install,...,"(fire_IFLT,)","(fire_NONE,)","(fire_SUBS,)","(fire_nan,)","(expl_BOTH,)","(expl_GRD ,)","(expl_IFLT,)","(expl_NONE,)","(expl_SUBS,)","(expl_nan,)"
0,0.0,0,Cessna,207,3800.0,AIR,N,2.0,1.0,Y,...,0,1,0,0,0,0,0,1,0,0
1,0.0,0,Boeing,747-100,750000.0,AIR,N,9.0,4.0,Y,...,0,0,0,0,0,0,0,1,0,0
2,0.0,0,Piper,PA-31-350,7369.0,AIR,N,6.0,2.0,Y,...,0,1,0,0,0,0,0,1,0,0
3,0.0,0,Cessna,172,2300.0,AIR,N,4.0,1.0,Y,...,0,1,0,0,0,0,0,1,0,0
4,0.0,0,Cessna,207,3800.0,AIR,N,6.0,1.0,Y,...,0,1,0,0,0,0,0,1,0,0


##### Aircraft Make |  `acft_make` 

In [53]:
ntsb08['acft_make'].value_counts().sort_values(ascending=False)

CESSNA                            13583
PIPER                              7368
Cessna                             4248
Cessna                             3306
BEECH                              2352
                                  ...  
YORK                                  1
HERINK H R/HERINK R W                 1
LOWTHER                               1
SCHICORA                              1
Bair Helicopters, Inc.                1
Name: acft_make, Length: 5442, dtype: int64

**Comment:** This column requires additional cleaning where all the text will be capitalized and column is split to remove spaces after the text. 

In [54]:
makedf = ntsb08['acft_make'].str.split(' ', expand=True)
makedf

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,Cessna,,,,,,,,,,...,,,,,None,None,None,None,None,None
1,Boeing,,,,,,,,,,...,,,,,None,None,None,None,None,None
2,Piper,,,,,,,,,,...,,,,,,None,None,None,None,None
3,Cessna,,,,,,,,,,...,,,,,None,None,None,None,None,None
4,Cessna,,,,,,,,,,...,,,,,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
makedf[0] = makedf[0].str.upper()
makedf.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,CESSNA,,,,,,,,,,...,,,,,None,None,None,None,None,None
1,BOEING,,,,,,,,,,...,,,,,None,None,None,None,None,None
2,PIPER,,,,,,,,,,...,,,,,,None,None,None,None,None
3,CESSNA,,,,,,,,,,...,,,,,None,None,None,None,None,None
4,CESSNA,,,,,,,,,,...,,,,,None,None,None,None,None,None


In [56]:
ntsb08.insert(3, 'make', makedf[0])
ntsb08.drop(['acft_make'], axis=1)
ntsb08.head()

,acft_missing,flight_plan_activated,acft_make,make,acft_model,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,...,"(fire_IFLT,)","(fire_NONE,)","(fire_SUBS,)","(fire_nan,)","(expl_BOTH,)","(expl_GRD ,)","(expl_IFLT,)","(expl_NONE,)","(expl_SUBS,)","(expl_nan,)"
0,0.0,0,Cessna,CESSNA,207,3800.0,AIR,N,2.0,1.0,...,0,1,0,0,0,0,0,1,0,0
1,0.0,0,Boeing,BOEING,747-100,750000.0,AIR,N,9.0,4.0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,0,Piper,PIPER,PA-31-350,7369.0,AIR,N,6.0,2.0,...,0,1,0,0,0,0,0,1,0,0
3,0.0,0,Cessna,CESSNA,172,2300.0,AIR,N,4.0,1.0,...,0,1,0,0,0,0,0,1,0,0
4,0.0,0,Cessna,CESSNA,207,3800.0,AIR,N,6.0,1.0,...,0,1,0,0,0,0,0,1,0,0


In [57]:
ntsb08['make'].value_counts().sort_values(ascending=False)

CESSNA          21228
PIPER           11457
BEECH            4124
BELL             1957
BOEING           1433
                ...  
AERODYNAMICS        1
LORANGER            1
WEAVER,             1
MULHOLLAND          1
BAIR                1
Name: make, Length: 3674, dtype: int64

In [58]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['make'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_make = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_make.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_make = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_make = encoded_make.add_prefix('make_')

# Add original back in (just to check)
encoded_make['acft_make'] = ntsb08['make']

# Show
encoded_make.head()

[array(['', '1200', '1977', ..., 'ZUKOWSKI', 'ZWART', nan], dtype=object)]


,make_,make_1200,make_1977,make_1ST,make_2000,make_2001,make_2003,make_67,make_85,make_A,...,make_ZILZ,make_ZIMMERMAN,make_ZITO,make_ZIVKO,make_ZLIN,make_ZORN,make_ZUKOWSKI,make_ZWART,make_nan,acft_make
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CESSNA
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BOEING
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,PIPER
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CESSNA
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CESSNA


In [59]:
#Drop the final column
encoded_make = encoded_make.drop(['acft_make'], axis=1)

#View percentage across columns
encoded_make.sum().sort_values(ascending=False)

make_CESSNA       21228
make_PIPER        11457
make_BEECH         4124
make_BELL          1957
make_BOEING        1433
                  ...  
make_HERRING          1
make_HERRON           1
make_HESTERLEE        1
make_HEVERON          1
make_KOJIMA,          1
Length: 3675, dtype: int64

In [60]:
#Drop 1x column to avoid collinearity
encoded_make = encoded_make.drop(['make_ZWART'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_make], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['make'], axis=1)
ntsb08 = ntsb08.drop(['acft_make'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,acft_model,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,elt_install,elt_oper,...,"(make_ZENITH,)","(make_ZIELKE,)","(make_ZILZ,)","(make_ZIMMERMAN,)","(make_ZITO,)","(make_ZIVKO,)","(make_ZLIN,)","(make_ZORN,)","(make_ZUKOWSKI,)","(make_nan,)"
0,0.0,0,207,3800.0,AIR,N,2.0,1.0,Y,Y,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,747-100,750000.0,AIR,N,9.0,4.0,Y,U,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,PA-31-350,7369.0,AIR,N,6.0,2.0,Y,N,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,172,2300.0,AIR,N,4.0,1.0,Y,N,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,207,3800.0,AIR,N,6.0,1.0,Y,Y,...,0,0,0,0,0,0,0,0,0,0


##### Aircraft Model |  `acft_model` 

In [61]:
ntsb08['acft_model'].value_counts().sort_values(ascending=False)

152                     1881
172                      881
172N                     773
PA-28-140                659
150                      607
                        ... 
KITFOX 2                   1
C18S                       1
SR8C                       1
MARK 5                     1
PA-32 300C                 1
Name: acft_model, Length: 10141, dtype: int64

In [62]:
modeldf = ntsb08['acft_model'].str.split(' ', expand=True)
modeldf[0] = modeldf[0].str.upper()
ntsb08.insert(4, 'model', modeldf[0])
ntsb08 = ntsb08.drop(['acft_model'], axis=1)
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,model,acft_category,homebuilt,total_seats,num_eng,elt_install,elt_oper,...,"(make_ZENITH,)","(make_ZIELKE,)","(make_ZILZ,)","(make_ZIMMERMAN,)","(make_ZITO,)","(make_ZIVKO,)","(make_ZLIN,)","(make_ZORN,)","(make_ZUKOWSKI,)","(make_nan,)"
0,0.0,0,3800.0,207,AIR,N,2.0,1.0,Y,Y,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,747-100,AIR,N,9.0,4.0,Y,U,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,PA-31-350,AIR,N,6.0,2.0,Y,N,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,172,AIR,N,4.0,1.0,Y,N,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,207,AIR,N,6.0,1.0,Y,Y,...,0,0,0,0,0,0,0,0,0,0


In [63]:
ntsb08['model'].value_counts().sort_values(ascending=False)

152           2187
172           1072
172N          1012
PA-28-140      816
150            673
              ... 
T-6-SNJ-5        1
PA60-602P        1
E-2              1
181              1
CRAIGCRAFT       1
Name: model, Length: 6303, dtype: int64

In [64]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['model'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_model = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_model.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_model = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_model = encoded_model.add_prefix('model_')

# Add original back in (just to check)
encoded_model['acft_model'] = ntsb08['model']

# Show
encoded_model.head()

[array(['', '&GCBC', '(MODIFIED)', ..., 'ZUGVOGEL', 'ZUNI', nan],
      dtype=object)]


,model_,model_&GCBC,model_(MODIFIED),model_(SOLOY,model_0-1A,model_0-47B,model_0-58B,model_0-77,model_001,model_01,...,model_ZIPPPY,model_ZIPPY,model_ZKC-5,model_ZLIN,model_ZMI-A,model_ZODIAC,model_ZUGVOGEL,model_ZUNI,model_nan,acft_model
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,207
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,747-100
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,PA-31-350
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,172
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,207


In [65]:
#Drop the final column
encoded_model = encoded_model.drop(['acft_model'], axis=1)

#View total count across columns
encoded_model.sum().sort_values(ascending=False)

model_152               2187
model_172               1072
model_172N              1012
model_PA-28-140          816
model_150                673
                        ... 
model_FLUT-R-BUG           1
model_FLY-BABY             1
model_FN333                1
model_FOCKE-WULF-190       1
model_HP-19C               1
Length: 6304, dtype: int64

In [66]:
#Drop 1x column to avoid collinearity
encoded_model = encoded_model.drop(['model_F27-100'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_model], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['model'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,acft_category,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,...,"(model_ZIC,)","(model_ZIPPPY,)","(model_ZIPPY,)","(model_ZKC-5,)","(model_ZLIN,)","(model_ZMI-A,)","(model_ZODIAC,)","(model_ZUGVOGEL,)","(model_ZUNI,)","(model_nan,)"
0,0.0,0,3800.0,AIR,N,2.0,1.0,Y,Y,U,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,AIR,N,9.0,4.0,Y,U,U,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,AIR,N,6.0,2.0,Y,N,U,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,AIR,N,4.0,1.0,Y,N,U,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,AIR,N,6.0,1.0,Y,Y,N,...,0,0,0,0,0,0,0,0,0,0


##### Aircraft Category |  `acft_category` 

In [67]:
ntsb08['acft_category'].value_counts().sort_values(ascending=False)

AIR     56316
HELI     5664
GLI      1068
BALL      545
GYRO      283
ULTR      254
BLIM       21
UNK        19
PPAR        3
PLFT        3
AIR         3
WSFT        1
Name: acft_category, dtype: int64

In [68]:
#Replace UNK 
ntsb08['acft_category'] = np.where(ntsb08['acft_category'] == 'UNK', 'AIR', ntsb08['acft_category'])
ntsb08['acft_category'] = np.where(ntsb08['acft_category'] == 'UNK ', 'AIR', ntsb08['acft_category'])
#Verify
ntsb08['acft_category'].value_counts().sort_values(ascending=False)

AIR     56316
HELI     5664
GLI      1068
BALL      545
GYRO      283
ULTR      254
AIR        22
BLIM       21
PPAR        3
PLFT        3
WSFT        1
Name: acft_category, dtype: int64

In [69]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['acft_category'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_cat = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_cat.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_cat = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_cat = encoded_cat.add_prefix('cat_')

# Add original back in (just to check)
encoded_cat['acft_cat'] = ntsb08['acft_category']

# Show
encoded_cat.head()

[array(['AIR', 'AIR ', 'BALL', 'BLIM', 'GLI ', 'GYRO', 'HELI', 'PLFT',
       'PPAR', 'ULTR', 'WSFT', nan], dtype=object)]


,cat_AIR,cat_AIR,cat_BALL,cat_BLIM,cat_GLI,cat_GYRO,cat_HELI,cat_PLFT,cat_PPAR,cat_ULTR,cat_WSFT,cat_nan,acft_cat
0,0,1,0,0,0,0,0,0,0,0,0,0,AIR
1,0,1,0,0,0,0,0,0,0,0,0,0,AIR
2,0,1,0,0,0,0,0,0,0,0,0,0,AIR
3,0,1,0,0,0,0,0,0,0,0,0,0,AIR
4,0,1,0,0,0,0,0,0,0,0,0,0,AIR


In [70]:
#Drop the final column
encoded_cat = encoded_cat.drop(['acft_cat'], axis=1)

#View total count across columns
encoded_cat.sum().sort_values(ascending=False)

cat_AIR     56316
cat_HELI     5664
cat_GLI      1068
cat_BALL      545
cat_GYRO      283
cat_ULTR      254
cat_AIR        22
cat_BLIM       21
cat_nan        12
cat_PLFT        3
cat_PPAR        3
cat_WSFT        1
dtype: int64

In [71]:
#Drop 1x column to avoid collinearity
encoded_cat = encoded_cat.drop(['cat_WSFT'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_cat], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['acft_category'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(cat_AIR ,)","(cat_BALL,)","(cat_BLIM,)","(cat_GLI ,)","(cat_GYRO,)","(cat_HELI,)","(cat_PLFT,)","(cat_PPAR,)","(cat_ULTR,)","(cat_nan,)"
0,0.0,0,3800.0,N,2.0,1.0,Y,Y,U,Y,...,1,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,N,9.0,4.0,Y,U,U,Y,...,1,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,N,6.0,2.0,Y,N,U,Y,...,1,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,N,4.0,1.0,Y,N,U,N,...,1,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,N,6.0,1.0,Y,Y,N,Y,...,1,0,0,0,0,0,0,0,0,0


##### Aircraft Make |  `homebuilt` 

In [72]:
ntsb08['homebuilt'].value_counts()

N      58508
Y       5438
U        234
Name: homebuilt, dtype: int64

In [73]:
ntsb08['homebuilt'].unique()

array(['N  ', 'Y  ', 'U  ', nan], dtype=object)

In [74]:
ntsb08['homebuilt'] = np.where(ntsb08['homebuilt'] == 'Y  ', 1, 0)
ntsb08['homebuilt'].value_counts()

0    58754
1     5438
Name: homebuilt, dtype: int64

##### ELT Installed | `elt_install`

In [75]:
ntsb08['elt_install'].value_counts()

Y    45390
N    12189
U     6601
Name: elt_install, dtype: int64

In [76]:
ntsb08['elt_install'] = np.where(ntsb08['elt_install'] == 'Y', 1, 0)
ntsb08['elt_install'].value_counts()

1    45390
0    18802
Name: elt_install, dtype: int64

##### ELT Operational | `elt_oper`

In [77]:
ntsb08['elt_oper'].value_counts()

N    29910
U    19560
Y    14710
Name: elt_oper, dtype: int64

In [78]:
ntsb08['elt_oper'] = np.where(ntsb08['elt_oper'] == 'Y', 1, 0)
ntsb08['elt_oper'].value_counts()

0    49482
1    14710
Name: elt_oper, dtype: int64

##### ELT Aided Locating Aircraft | `elt_aided_loc_ev`

In [79]:
ntsb08['elt_aided_loc_ev'].value_counts()

N    32712
U    29337
Y     2131
Name: elt_aided_loc_ev, dtype: int64

In [80]:
ntsb08['elt_aided_loc_ev'] = np.where(ntsb08['elt_aided_loc_ev'] == 'Y', 1, 0)
ntsb08['elt_aided_loc_ev'].value_counts()

0    62061
1     2131
Name: elt_aided_loc_ev, dtype: int64

##### Appropriate Certifications Held (aircraft) | `certs_held`

In [81]:
ntsb08['certs_held'].value_counts()

N    49804
Y    14376
Name: certs_held, dtype: int64

In [82]:
ntsb08['certs_held'] = np.where(ntsb08['certs_held'] == 'Y', 1, 0)
ntsb08['certs_held'].value_counts()

0    49816
1    14376
Name: certs_held, dtype: int64

##### Reason for Flying | `type_fly`

In [83]:
ntsb08['type_fly'].value_counts()

PERS    37534
INST     8716
UNK      5939
AAPL     3638
BUS      3433
POSI     1257
OWRK      928
PUBU      691
FERY      676
AOBV      471
EXEC      464
FLTS      182
SKYD       71
ASHO       61
PUBF       28
BANT       26
EXLD       16
PUBS       16
GLDT       12
PUBL       11
ADRP        5
FIRF        5
Name: type_fly, dtype: int64

In [84]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['type_fly'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded_typef = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded_typef.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded_typef = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded_typef = encoded_typef.add_prefix('typefly_')

# Add original back in (just to check)
encoded_typef['type_fly'] = ntsb08['type_fly']

# Show
encoded_typef.head()

[array(['AAPL', 'ADRP', 'AOBV', 'ASHO', 'BANT', 'BUS ', 'EXEC', 'EXLD',
       'FERY', 'FIRF', 'FLTS', 'GLDT', 'INST', 'OWRK', 'PERS', 'POSI',
       'PUBF', 'PUBL', 'PUBS', 'PUBU', 'SKYD', 'UNK ', nan], dtype=object)]


,typefly_AAPL,typefly_ADRP,typefly_AOBV,typefly_ASHO,typefly_BANT,typefly_BUS,typefly_EXEC,typefly_EXLD,typefly_FERY,typefly_FIRF,...,typefly_PERS,typefly_POSI,typefly_PUBF,typefly_PUBL,typefly_PUBS,typefly_PUBU,typefly_SKYD,typefly_UNK,typefly_nan,type_fly
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,UNK
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,UNK
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,UNK
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,PERS
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,UNK


In [85]:
#Drop the final column
encoded_typef = encoded_typef.drop(['type_fly'], axis=1)

#View total count across columns
encoded_typef.sum().sort_values(ascending=False)

typefly_PERS    37534
typefly_INST     8716
typefly_UNK      5939
typefly_AAPL     3638
typefly_BUS      3433
typefly_POSI     1257
typefly_OWRK      928
typefly_PUBU      691
typefly_FERY      676
typefly_AOBV      471
typefly_EXEC      464
typefly_FLTS      182
typefly_SKYD       71
typefly_ASHO       61
typefly_PUBF       28
typefly_BANT       26
typefly_EXLD       16
typefly_PUBS       16
typefly_GLDT       12
typefly_nan        12
typefly_PUBL       11
typefly_ADRP        5
typefly_FIRF        5
dtype: int64

In [86]:
#Drop 1x column to avoid collinearity
encoded_typef = encoded_typef.drop(['typefly_FIRF'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded_typef], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['type_fly'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(typefly_OWRK,)","(typefly_PERS,)","(typefly_POSI,)","(typefly_PUBF,)","(typefly_PUBL,)","(typefly_PUBS,)","(typefly_PUBU,)","(typefly_SKYD,)","(typefly_UNK ,)","(typefly_nan,)"
0,0.0,0,3800.0,0,2.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,0.0,0,750000.0,0,9.0,4.0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0,7369.0,0,6.0,2.0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0,2300.0,0,4.0,1.0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,0,6.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0


##### Second Pilot | `second_pilot`

In [87]:
ntsb08['second_pilot'].value_counts()

N    50055
Y    11256
U     2869
Name: second_pilot, dtype: int64

In [88]:
ntsb08['second_pilot'] = np.where(ntsb08['second_pilot'] == 'Y', 1, 0)
ntsb08['second_pilot'].value_counts()

0    52936
1    11256
Name: second_pilot, dtype: int64

##### Site Seeing Operations | `site_seeing`

In [89]:
ntsb08['site_seeing'].value_counts()

N    63726
Y      454
Name: site_seeing, dtype: int64

In [90]:
ntsb08['site_seeing'] = np.where(ntsb08['site_seeing'] == 'Y', 1, 0)
ntsb08['site_seeing'].value_counts()

0    63738
1      454
Name: site_seeing, dtype: int64

##### Air Medical Operation | `air_medical`

In [91]:
ntsb08['air_medical'].value_counts()

N    63914
Y      266
Name: air_medical, dtype: int64

In [92]:
ntsb08['air_medical'] = np.where(ntsb08['air_medical'] == 'Y', 1, 0)
ntsb08['air_medical'].value_counts()

0    63926
1      266
Name: air_medical, dtype: int64

<a id='dprt-info'></a>
### Departure Information Categorical Features
#### Departure State | `dprt_state`

In [93]:
ntsb08['dprt_state'] = ntsb08['dprt_state'].str.upper()

In [94]:
ntsb08['dprt_state'].value_counts()

      17544
CA     5328
AK     3214
TX     3155
FL     3114
      ...  
ON        1
IO        1
FR        1
JA        1
UK        1
Name: dprt_state, Length: 84, dtype: int64

In [95]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['dprt_state'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('dprtstate_')

# Add original back in (just to check)
encoded['dprt_state'] = ntsb08['dprt_state']

# Show
encoded.head()

[array(['  ', 'A ', 'AD', 'AH', 'AK', 'AL', 'AR', 'AZ', 'BC', 'BH', 'BI',
       'CA', 'CB', 'CD', 'CN', 'CO', 'CT', 'DC', 'DE', 'DR', 'FL', 'FR',
       'GA', 'GM', 'GU', 'HA', 'HI', 'I ', 'IA', 'ID', 'IL', 'IN', 'IO',
       'IT', 'JA', 'JP', 'KN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MH',
       'MI', 'MN', 'MO', 'MS', 'MT', 'MX', 'NB', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OF', 'OH', 'OK', 'ON', 'OR', 'PA', 'PO',
       'PQ', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UK', 'UN', 'UT', 'VA',
       'VI', 'VT', 'WA', 'WI', 'WV', 'WY', 'YT', nan], dtype=object)]


,dprtstate_,dprtstate_A,dprtstate_AD,dprtstate_AH,dprtstate_AK,dprtstate_AL,dprtstate_AR,dprtstate_AZ,dprtstate_BC,dprtstate_BH,...,dprtstate_VA,dprtstate_VI,dprtstate_VT,dprtstate_WA,dprtstate_WI,dprtstate_WV,dprtstate_WY,dprtstate_YT,dprtstate_nan,dprt_state
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,JA
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK


In [96]:
#Drop the final column
encoded= encoded.drop(['dprt_state'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

dprtstate_      17544
dprtstate_CA     5328
dprtstate_AK     3214
dprtstate_TX     3155
dprtstate_FL     3114
                ...  
dprtstate_HA        1
dprtstate_PQ        1
dprtstate_A         1
dprtstate_DR        1
dprtstate_MH        1
Length: 85, dtype: int64

In [97]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['dprtstate_MH'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['dprt_state'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(dprtstate_UT,)","(dprtstate_VA,)","(dprtstate_VI,)","(dprtstate_VT,)","(dprtstate_WA,)","(dprtstate_WI,)","(dprtstate_WV,)","(dprtstate_WY,)","(dprtstate_YT,)","(dprtstate_nan,)"
0,0.0,0,3800.0,0,2.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,0,9.0,4.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,0,6.0,2.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,0,4.0,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,0,6.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


##### Departure Country | `dprt_country`

In [98]:
ntsb08['dprt_country'].value_counts()

       48634
USA    15388
CA        23
BF        22
MX        21
UK         9
UY         9
JA         9
IT         7
GE         6
HK         4
FR         4
CH         4
CO         4
EI         3
BR         3
TW         2
JM         2
AS         2
NL         2
BE         2
UZ         2
BB         2
BD         2
SC         2
TD         2
NH         2
PE         2
RP         1
DR         1
VE         1
TK         1
CU         1
AC         1
Name: dprt_country, dtype: int64

In [99]:
ntsb08['dprt_country'] = ntsb08['dprt_country'].str.upper()

In [100]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['dprt_country'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('dprtctry_')

# Add original back in (just to check)
encoded['dprt_country'] = ntsb08['dprt_country']

# Show
encoded.head()

[array(['   ', 'AC ', 'AS ', 'BB ', 'BD ', 'BE ', 'BF ', 'BR ', 'CA ',
       'CH ', 'CO ', 'CU ', 'DR ', 'EI ', 'FR ', 'GE ', 'HK ', 'IT ',
       'JA ', 'JM ', 'MX ', 'NH ', 'NL ', 'PE ', 'RP ', 'SC ', 'TD ',
       'TK ', 'TW ', 'UK ', 'USA', 'UY ', 'UZ ', 'VE ', nan], dtype=object)]


,dprtctry_,dprtctry_AC,dprtctry_AS,dprtctry_BB,dprtctry_BD,dprtctry_BE,dprtctry_BF,dprtctry_BR,dprtctry_CA,dprtctry_CH,...,dprtctry_TD,dprtctry_TK,dprtctry_TW,dprtctry_UK,dprtctry_USA,dprtctry_UY,dprtctry_UZ,dprtctry_VE,dprtctry_nan,dprt_country
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,


In [101]:
#Drop the final column
encoded= encoded.drop(['dprt_country'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

dprtctry_       48634
dprtctry_USA    15388
dprtctry_CA        23
dprtctry_BF        22
dprtctry_MX        21
dprtctry_nan       12
dprtctry_JA         9
dprtctry_UK         9
dprtctry_UY         9
dprtctry_IT         7
dprtctry_GE         6
dprtctry_HK         4
dprtctry_FR         4
dprtctry_CO         4
dprtctry_CH         4
dprtctry_EI         3
dprtctry_BR         3
dprtctry_PE         2
dprtctry_AS         2
dprtctry_UZ         2
dprtctry_TW         2
dprtctry_TD         2
dprtctry_SC         2
dprtctry_BE         2
dprtctry_NL         2
dprtctry_NH         2
dprtctry_JM         2
dprtctry_BD         2
dprtctry_BB         2
dprtctry_RP         1
dprtctry_TK         1
dprtctry_AC         1
dprtctry_DR         1
dprtctry_CU         1
dprtctry_VE         1
dtype: int64

In [102]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['dprtctry_VE '], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['dprt_country'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(dprtctry_RP ,)","(dprtctry_SC ,)","(dprtctry_TD ,)","(dprtctry_TK ,)","(dprtctry_TW ,)","(dprtctry_UK ,)","(dprtctry_USA,)","(dprtctry_UY ,)","(dprtctry_UZ ,)","(dprtctry_nan,)"
0,0.0,0,3800.0,0,2.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,0,9.0,4.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,0,6.0,2.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,0,4.0,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,0,6.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


<a id='dest-info'></a>
### Destination Information Categorical Featues
##### Destination State | `dest_state`

In [103]:
ntsb08['dest_state'] = ntsb08['dest_state'].str.upper()

In [104]:
ntsb08['dest_state'].value_counts()

      28462
CA     8361
AK     2238
FL     2124
TX     2088
      ...  
         1
GE        1
2         1
VN        1
CM        1
Name: dest_state, Length: 108, dtype: int64

In [105]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['dest_state'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('dststate_')

# Add original back in (just to check)
encoded['dest_state'] = ntsb08['dest_state']

# Show
encoded.head()

[array(['\x02 ', '\x11 ', '  ', ' A', '0 ', '2 ', '42', '4G', 'A ', 'AK',
       'AL', 'AO', 'AP', 'AR', 'AZ', 'BC', 'BM', 'BU', 'CA', 'CB', 'CD',
       'CG', 'CK', 'CL', 'CM', 'CN', 'CO', 'CT', 'CU', 'D ', 'DC', 'DE',
       'DL', 'EN', 'FD', 'FI', 'FL', 'FM', 'FO', 'GA', 'GC', 'GE', 'GM',
       'GO', 'GU', 'HI', 'HL', 'HW', 'IA', 'ID', 'IL', 'IN', 'IT', 'JA',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT',
       'MX', 'NB', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NL', 'NM', 'NO', 'NS',
       'NV', 'NY', 'NZ', 'OF', 'OH', 'OK', 'ON', 'OR', 'P1', 'PA', 'PO',
       'PQ', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UK', 'UN', 'UT', 'VA',
       'VI', 'VN', 'VT', 'WA', 'WI', 'WS', 'WV', 'WY', 'YT', 'ŸŸ', nan],
      dtype=object)]


,dststate_,dststate_,dststate_,dststate_ A,dststate_0,dststate_2,dststate_42,dststate_4G,dststate_A,dststate_AK,...,dststate_VT,dststate_WA,dststate_WI,dststate_WS,dststate_WV,dststate_WY,dststate_YT,dststate_ŸŸ,dststate_nan,dest_state
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,


In [106]:
#Drop the final column
encoded= encoded.drop(['dest_state'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

dststate_      28462
dststate_CA     8361
dststate_AK     2238
dststate_FL     2124
dststate_TX     2088
               ...  
dststate_DL        1
dststate_D         1
dststate_CU        1
dststate_JA        1
dststate_         1
Length: 109, dtype: int64

In [107]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['dststate_JA'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['dest_state'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(dststate_VN,)","(dststate_VT,)","(dststate_WA,)","(dststate_WI,)","(dststate_WS,)","(dststate_WV,)","(dststate_WY,)","(dststate_YT,)","(dststate_ŸŸ,)","(dststate_nan,)"
0,0.0,0,3800.0,0,2.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,0,9.0,4.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,0,6.0,2.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,0,4.0,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,0,6.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


##### Destination Country | `dest_country`

In [108]:
ntsb08['dest_country'] = ntsb08['dest_country'].str.upper()

In [109]:
ntsb08['dest_country'].value_counts()

       48634
USA    15384
CA        29
BF        20
JA        13
MX        12
UK        10
CH         6
GE         6
BR         5
SP         4
IC         4
FR         4
AS         4
UN         3
FJ         3
TW         3
NL         3
DR         3
HA         2
PM         2
HR         2
EG         2
EI         2
SZ         2
AA         2
RP         2
KS         2
IT         2
CJ         1
NH         1
RS         1
TU         1
UZ         1
PS         1
TK         1
VI         1
VM         1
UY         1
Name: dest_country, dtype: int64

In [110]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['dest_country'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('dstctry_')

# Add original back in (just to check)
encoded['dest_country'] = ntsb08['dest_country']

# Show
encoded.head()

[array(['   ', 'AA ', 'AS ', 'BF ', 'BR ', 'CA ', 'CH ', 'CJ ', 'DR ',
       'EG ', 'EI ', 'FJ ', 'FR ', 'GE ', 'HA ', 'HR ', 'IC ', 'IT ',
       'JA ', 'KS ', 'MX ', 'NH ', 'NL ', 'PM ', 'PS ', 'RP ', 'RS ',
       'SP ', 'SZ ', 'TK ', 'TU ', 'TW ', 'UK ', 'UN ', 'USA', 'UY ',
       'UZ ', 'VI ', 'VM ', nan], dtype=object)]


,dstctry_,dstctry_AA,dstctry_AS,dstctry_BF,dstctry_BR,dstctry_CA,dstctry_CH,dstctry_CJ,dstctry_DR,dstctry_EG,...,dstctry_TW,dstctry_UK,dstctry_UN,dstctry_USA,dstctry_UY,dstctry_UZ,dstctry_VI,dstctry_VM,dstctry_nan,dest_country
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,


In [111]:
#Drop the final column
encoded= encoded.drop(['dest_country'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

dstctry_       48634
dstctry_USA    15384
dstctry_CA        29
dstctry_BF        20
dstctry_JA        13
dstctry_MX        12
dstctry_nan       12
dstctry_UK        10
dstctry_GE         6
dstctry_CH         6
dstctry_BR         5
dstctry_FR         4
dstctry_AS         4
dstctry_IC         4
dstctry_SP         4
dstctry_NL         3
dstctry_UN         3
dstctry_DR         3
dstctry_TW         3
dstctry_FJ         3
dstctry_EI         2
dstctry_SZ         2
dstctry_RP         2
dstctry_IT         2
dstctry_KS         2
dstctry_EG         2
dstctry_HA         2
dstctry_HR         2
dstctry_PM         2
dstctry_AA         2
dstctry_TK         1
dstctry_TU         1
dstctry_PS         1
dstctry_RS         1
dstctry_NH         1
dstctry_UY         1
dstctry_UZ         1
dstctry_VI         1
dstctry_VM         1
dstctry_CJ         1
dtype: int64

In [112]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['dstctry_VM '], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['dest_country'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(dstctry_TK ,)","(dstctry_TU ,)","(dstctry_TW ,)","(dstctry_UK ,)","(dstctry_UN ,)","(dstctry_USA,)","(dstctry_UY ,)","(dstctry_UZ ,)","(dstctry_VI ,)","(dstctry_nan,)"
0,0.0,0,3800.0,0,2.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0,750000.0,0,9.0,4.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0,7369.0,0,6.0,2.0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0,2300.0,0,4.0,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0,3800.0,0,6.0,1.0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


<a id='ev-info'></a>
### Event Information Categorical Features
##### Event Type | `ev_type`

In [113]:
ntsb08['ev_type'].value_counts()

ACC    62005
INC     2175
Name: ev_type, dtype: int64

In [114]:
ntsb08['ev_type'] = np.where(ntsb08['ev_type'] == 'ACC', 1, 0)
ntsb08['ev_type'].value_counts()

1    62005
0     2187
Name: ev_type, dtype: int64

##### Event Date | `ev_date`

In [115]:
ntsb08['ev_date'].head()

0    1999-01-10
1    1999-01-19
2    1999-01-05
3    1999-01-10
4    1999-01-12
Name: ev_date, dtype: object

**Comment:** When investigating the event date feature, the data type should be changed. In addition, there are already columns for the event year and month. This is redundant, the numerical day will be spliced from the current event date feature and added back into the main dataframe.

In [116]:
ev_date = ntsb08['ev_date'].str.split('-', expand=True)
ev_date.head()

,0,1,2
0,1999,01,10
1,1999,01,19
2,1999,01,05
3,1999,01,10
4,1999,01,12


In [117]:
ntsb08.columns.get_loc('ev_date')

20

In [118]:
ntsb08.insert(20,'ev_day', ev_date[2])
ntsb08 = ntsb08.drop(['ev_date'], axis=1)

In [119]:
ntsb08 = ntsb08.dropna()

In [120]:
ntsb08['ev_day'] = ntsb08['ev_day'].astype(int)

##### Event Day of the Week | `ev_dow`
This is a redundant feature and will be dropped.

In [121]:
ntsb08 = ntsb08.drop(['ev_dow'], axis=1)

##### Event Time Zone | `ev_tmzn`

In [122]:
ntsb08['ev_tmzn'] = ntsb08['ev_tmzn'].str.upper()
ntsb08['ev_tmzn'].value_counts()

EDT    12555
CDT    12004
EST     7865
PDT     7500
CST     5966
MDT     4567
MST     4326
PST     4244
AKD     3278
AST     1278
HST      391
         115
UTC       37
GMT       11
YST        7
HDT        4
TL         4
ÜÇN        3
BST        2
ADT        2
TO         2
TW         2
TD         1
LMT        1
GST        1
TN         1
YDT        1
Name: ev_tmzn, dtype: int64

In [123]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['ev_tmzn'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('evtmzn_')

# Add original back in (just to check)
encoded['ev_tmzn'] = ntsb08['ev_tmzn']

# Show
encoded.head()

[array(['   ', 'ADT', 'AKD', 'AST', 'BST', 'CDT', 'CST', 'EDT', 'EST',
       'GMT', 'GST', 'HDT', 'HST', 'LMT', 'MDT', 'MST', 'PDT', 'PST',
       'TD', 'TL', 'TN', 'TO', 'TW', 'UTC', 'YDT', 'YST', 'ÜÇN'],
      dtype=object)]


,evtmzn_,evtmzn_ADT,evtmzn_AKD,evtmzn_AST,evtmzn_BST,evtmzn_CDT,evtmzn_CST,evtmzn_EDT,evtmzn_EST,evtmzn_GMT,...,evtmzn_TD,evtmzn_TL,evtmzn_TN,evtmzn_TO,evtmzn_TW,evtmzn_UTC,evtmzn_YDT,evtmzn_YST,evtmzn_ÜÇN,ev_tmzn
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AST
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AST
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AST
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AST
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AST


In [124]:
#Drop the final column
encoded= encoded.drop(['ev_tmzn'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

evtmzn_EDT    12555
evtmzn_CDT    12004
evtmzn_EST     7865
evtmzn_PDT     7500
evtmzn_CST     5966
evtmzn_MDT     4567
evtmzn_MST     4326
evtmzn_PST     4244
evtmzn_AKD     3278
evtmzn_AST     1278
evtmzn_HST      391
evtmzn_         115
evtmzn_UTC       37
evtmzn_GMT       11
evtmzn_YST        7
evtmzn_HDT        4
evtmzn_TL         4
evtmzn_ÜÇN        3
evtmzn_ADT        2
evtmzn_TO         2
evtmzn_TW         2
evtmzn_BST        2
evtmzn_TD         1
evtmzn_TN         1
evtmzn_GST        1
evtmzn_YDT        1
evtmzn_LMT        1
dtype: int64

In [125]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['evtmzn_LMT'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['ev_tmzn'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(evtmzn_PST,)","(evtmzn_TD,)","(evtmzn_TL,)","(evtmzn_TN,)","(evtmzn_TO,)","(evtmzn_TW,)","(evtmzn_UTC,)","(evtmzn_YDT,)","(evtmzn_YST,)","(evtmzn_ÜÇN,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Event State | `ev_state`

In [126]:
ntsb08['ev_state'] = ntsb08['ev_state'].str.upper()
ntsb08['ev_state'].value_counts()

CA    7220
FL    4519
AK    4404
TX    4345
CO    2211
AZ    2130
WA    2031
MI    1663
IL    1659
NY    1528
GA    1489
OH    1417
PA    1389
OR    1363
AR    1229
MO    1229
NC    1212
WI    1211
MN    1128
NM    1051
IN    1014
ID     986
OK     979
UT     968
NJ     965
VA     954
LA     930
AL     906
NV     902
KS     848
MA     826
MT     808
TN     803
SC     713
MS     648
MD     607
IA     598
WY     564
NE     561
KY     496
ND     456
CT     414
HI     388
ME     381
SD     325
WV     316
NH     300
PR     215
VT     190
GM     168
RI     137
AO     100
DE      87
PO      69
DC      43
VI      42
CB      17
GU      12
UN       4
Name: ev_state, dtype: int64

In [127]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['ev_state'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('evstate_')

# Add original back in (just to check)
encoded['ev_state'] = ntsb08['ev_state']

# Show
encoded.head()

[array(['AK', 'AL', 'AO', 'AR', 'AZ', 'CA', 'CB', 'CO', 'CT', 'DC', 'DE',
       'FL', 'GA', 'GM', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY',
       'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PO',
       'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UN', 'UT', 'VA', 'VI', 'VT',
       'WA', 'WI', 'WV', 'WY', nan], dtype=object)]


,evstate_AK,evstate_AL,evstate_AO,evstate_AR,evstate_AZ,evstate_CA,evstate_CB,evstate_CO,evstate_CT,evstate_DC,...,evstate_UT,evstate_VA,evstate_VI,evstate_VT,evstate_WA,evstate_WI,evstate_WV,evstate_WY,evstate_nan,ev_state
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AK


In [128]:
#Drop the final column
encoded= encoded.drop(['ev_state'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

evstate_CA     7220
evstate_FL     4519
evstate_AK     4404
evstate_TX     4345
evstate_CO     2211
evstate_AZ     2130
evstate_WA     2031
evstate_MI     1663
evstate_IL     1659
evstate_NY     1528
evstate_GA     1489
evstate_OH     1417
evstate_PA     1389
evstate_OR     1363
evstate_MO     1229
evstate_AR     1229
evstate_NC     1212
evstate_WI     1211
evstate_MN     1128
evstate_NM     1051
evstate_IN     1014
evstate_ID      986
evstate_OK      979
evstate_UT      968
evstate_NJ      965
evstate_VA      954
evstate_LA      930
evstate_AL      906
evstate_NV      902
evstate_KS      848
evstate_MA      826
evstate_MT      808
evstate_TN      803
evstate_SC      713
evstate_MS      648
evstate_MD      607
evstate_IA      598
evstate_WY      564
evstate_NE      561
evstate_KY      496
evstate_ND      456
evstate_CT      414
evstate_HI      388
evstate_ME      381
evstate_SD      325
evstate_WV      316
evstate_NH      300
evstate_PR      215
evstate_VT      190
evstate_GM      168


In [129]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['evstate_UN'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['ev_state'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(evstate_TX,)","(evstate_UT,)","(evstate_VA,)","(evstate_VI,)","(evstate_VT,)","(evstate_WA,)","(evstate_WI,)","(evstate_WV,)","(evstate_WY,)","(evstate_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


##### Mid-Air Collision | `mid_air`

In [130]:
ntsb08['mid_air'] = ntsb08['mid_air'].str.upper()
ntsb08['mid_air'].value_counts()

N    63266
Y      902
Name: mid_air, dtype: int64

In [131]:
ntsb08['mid_air'] = np.where(ntsb08['mid_air'] == 'Y', 1, 0)

In [132]:
ntsb08['mid_air'].value_counts()

0    63278
1      902
Name: mid_air, dtype: int64

###### On-Ground Collision | `on_ground_collision`

In [133]:
ntsb08['on_ground_collision'] = ntsb08['on_ground_collision'].str.upper()
ntsb08['on_ground_collision'].value_counts()

N    62893
Y     1275
Name: on_ground_collision, dtype: int64

In [134]:
ntsb08['on_ground_collision'] = np.where(ntsb08['on_ground_collision'] == 'Y', 1, 0)

In [135]:
ntsb08['mid_air'].value_counts()

0    63278
1      902
Name: mid_air, dtype: int64

###### Nearest Airport to Event | `ev_nr_apt_loc`

In [136]:
ntsb08['ev_nr_apt_loc'].value_counts()

OFAP    34273
ONAP    28361
UNK      1534
Name: ev_nr_apt_loc, dtype: int64

In [137]:
print('Percent Unknown: ', ntsb08['ev_nr_apt_loc'].str.count('UNK').sum()/ntsb08.shape[0]*100)

Percent Unknown:  2.390152695543783


**Comment:** The category values include on or off airport (ONAP or OFAP respectively) only 2% of the data is 'Unknown' and will be rolled into the off airport (OFAP) category. ONAP will be coded as 1.

In [138]:
ntsb08['ev_nr_apt_loc'] = np.where(ntsb08['ev_nr_apt_loc'] == 'ONAP', 1, 0)
ntsb08['ev_nr_apt_loc'].value_counts()

0    35819
1    28361
Name: ev_nr_apt_loc, dtype: int64

<a id='wx-info'></a>
### Weather Information Categorical Features
#### Light Condition | `light_cond`

In [139]:
ntsb08['light_cond'] = ntsb08['light_cond'].str.upper()
ntsb08['light_cond'].value_counts()

DAYL    54141
NDRK     5102
DUSK     2116
NITE      874
NBRT      836
DAWN      662
NR        437
Name: light_cond, dtype: int64

In [140]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['light_cond'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('light_')

# Add original back in (just to check)
encoded['light_cond'] = ntsb08['light_cond']

# Show
encoded.head()

[array(['DAWN', 'DAYL', 'DUSK', 'NBRT', 'NDRK', 'NITE', 'NR', nan],
      dtype=object)]


,light_DAWN,light_DAYL,light_DUSK,light_NBRT,light_NDRK,light_NITE,light_NR,light_nan,light_cond
0,0,1,0,0,0,0,0,0,DAYL
1,0,0,0,0,1,0,0,0,NDRK
2,0,1,0,0,0,0,0,0,DAYL
3,0,1,0,0,0,0,0,0,DAYL
4,1,0,0,0,0,0,0,0,DAWN


In [141]:
#Drop the final column
encoded= encoded.drop(['light_cond'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

light_DAYL    54141
light_NDRK     5102
light_DUSK     2116
light_NITE      874
light_NBRT      836
light_DAWN      662
light_NR        437
light_nan        12
dtype: int64

In [142]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['light_NR'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['light_cond'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(evstate_WV,)","(evstate_WY,)","(evstate_nan,)","(light_DAWN,)","(light_DAYL,)","(light_DUSK,)","(light_NBRT,)","(light_NDRK,)","(light_NITE,)","(light_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,0,1,0,0,0,0,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,0,0,0,0


##### Sky Condition w/ No Ceiling | `sky_cond_nonceil`

In [143]:
ntsb08['sky_cond_nonceil'] = ntsb08['sky_cond_nonceil'].str.upper()
ntsb08['sky_cond_nonceil'].value_counts()

NONE    39232
BKN     11550
OVC      7463
UNK      4906
OBSC      940
VV         77
Name: sky_cond_nonceil, dtype: int64

**Comment:** Include unknowns in the 'NONE' category

In [144]:
ntsb08['sky_cond_nonceil'] = np.where(ntsb08['sky_cond_nonceil'] == 'UNK', 'NONE', ntsb08['sky_cond_nonceil'])
ntsb08['sky_cond_nonceil'].value_counts()

NONE    44138
BKN     11550
OVC      7463
OBSC      940
VV         77
Name: sky_cond_nonceil, dtype: int64

In [145]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['sky_cond_nonceil'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('skync_')

# Add original back in (just to check)
encoded['sky_cond_nonceil'] = ntsb08['sky_cond_nonceil']

# Show
encoded.head()

[array(['BKN', 'NONE', 'OBSC', 'OVC', 'VV', nan], dtype=object)]


,skync_BKN,skync_NONE,skync_OBSC,skync_OVC,skync_VV,skync_nan,sky_cond_nonceil
0,1,0,0,0,0,0,BKN
1,0,1,0,0,0,0,NONE
2,0,0,0,1,0,0,OVC
3,1,0,0,0,0,0,BKN
4,1,0,0,0,0,0,BKN


In [146]:
#Drop the final column
encoded= encoded.drop(['sky_cond_nonceil'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

skync_NONE    44138
skync_BKN     11550
skync_OVC      7463
skync_OBSC      940
skync_VV         77
skync_nan        12
dtype: int64

In [147]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['skync_VV'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['sky_cond_nonceil'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(light_DUSK,)","(light_NBRT,)","(light_NDRK,)","(light_NITE,)","(light_nan,)","(skync_BKN,)","(skync_NONE,)","(skync_OBSC,)","(skync_OVC,)","(skync_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,1,0,0,0,1,0,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,0


##### Sky Condition w/ Ceiling | `sky_cond_ceil`

In [148]:
ntsb08['sky_cond_ceil'] = ntsb08['sky_cond_ceil'].str.upper()
ntsb08['sky_cond_ceil'].value_counts()

NONE    39232
BKN     11550
OVC      7463
UNK      4906
OBSC      940
VV         77
Name: sky_cond_ceil, dtype: int64

**Comment:** Include unknowns in the 'NONE' category

In [149]:
ntsb08['sky_cond_ceil'] = np.where(ntsb08['sky_cond_ceil'] == 'UNK', 'NONE', ntsb08['sky_cond_ceil'])
ntsb08['sky_cond_ceil'].value_counts()

NONE    44138
BKN     11550
OVC      7463
OBSC      940
VV         77
Name: sky_cond_ceil, dtype: int64

In [150]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['sky_cond_ceil'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('skyc_')

# Add original back in (just to check)
encoded['sky_cond_ceil'] = ntsb08['sky_cond_ceil']

# Show
encoded.head()

[array(['BKN', 'NONE', 'OBSC', 'OVC', 'VV', nan], dtype=object)]


,skyc_BKN,skyc_NONE,skyc_OBSC,skyc_OVC,skyc_VV,skyc_nan,sky_cond_ceil
0,1,0,0,0,0,0,BKN
1,0,1,0,0,0,0,NONE
2,0,0,0,1,0,0,OVC
3,1,0,0,0,0,0,BKN
4,1,0,0,0,0,0,BKN


In [151]:
#Drop the final column
encoded= encoded.drop(['sky_cond_ceil'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

skyc_NONE    44138
skyc_BKN     11550
skyc_OVC      7463
skyc_OBSC      940
skyc_VV         77
skyc_nan        12
dtype: int64

In [152]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['skyc_VV'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['sky_cond_ceil'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(skync_BKN,)","(skync_NONE,)","(skync_OBSC,)","(skync_OVC,)","(skync_nan,)","(skyc_BKN,)","(skyc_NONE,)","(skyc_OBSC,)","(skyc_OVC,)","(skyc_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,1,0,0,0,0,1,0,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,1,0,0,0,0,1,0,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,1,0,0,0,0,1,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,1,0,0,0,0,1,0,0,0,0


##### Weather Condition | `wx_cond_basic`

In [153]:
ntsb08['wx_cond_basic'] = ntsb08['wx_cond_basic'].str.upper()
ntsb08['wx_cond_basic'].value_counts()

VMC    58679
IMC     4914
UNK      575
Name: wx_cond_basic, dtype: int64

**Comment:** Include unknowns as 'VMC'. 

In [154]:
ntsb08['wx_cond_basic'] = np.where(ntsb08['wx_cond_basic'] == 'UNK', 'VMC', ntsb08['wx_cond_basic'])
ntsb08['wx_cond_basic'].value_counts()

VMC    59254
IMC     4914
Name: wx_cond_basic, dtype: int64

In [155]:
ntsb08['wx_cond_basic'] = np.where(ntsb08['wx_cond_basic'] == 'VMC', 1, 0)
ntsb08['wx_cond_basic'].value_counts()

1    59254
0     4926
Name: wx_cond_basic, dtype: int64

<a id='crew-info'></a>
### Crew Information Categorical Features
##### Crew Category | `crew_category`

In [156]:
ntsb08['crew_category'] = ntsb08['crew_category'].str.upper()
ntsb08['crew_category'].value_counts()

PLT      60752
DSTU      1184
FLTI       917
CPLT       864
PLT        207
OTHR       135
KPLT        64
FENG        44
PRPS         1
Name: crew_category, dtype: int64

In [157]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['crew_category'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('crwcat_')

# Add original back in (just to check)
encoded['crew_category'] = ntsb08['crew_category']

# Show
encoded.head()

[array(['CPLT ', 'DSTU ', 'FENG ', 'FLTI ', 'KPLT ', 'OTHR ', 'PLT',
       'PLT  ', 'PRPS ', nan], dtype=object)]


,crwcat_CPLT,crwcat_DSTU,crwcat_FENG,crwcat_FLTI,crwcat_KPLT,crwcat_OTHR,crwcat_PLT,crwcat_PLT,crwcat_PRPS,crwcat_nan,crew_category
0,0,0,0,0,0,0,0,1,0,0,PLT
1,0,0,0,0,0,0,0,1,0,0,PLT
2,0,0,0,0,0,0,0,1,0,0,PLT
3,0,0,0,0,0,0,0,1,0,0,PLT
4,0,0,0,0,0,0,0,1,0,0,PLT


In [158]:
#Drop the final column
encoded= encoded.drop(['crew_category'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

crwcat_PLT      60752
crwcat_DSTU      1184
crwcat_FLTI       917
crwcat_CPLT       864
crwcat_PLT        207
crwcat_OTHR       135
crwcat_KPLT        64
crwcat_FENG        44
crwcat_nan         12
crwcat_PRPS         1
dtype: int64

In [159]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['crwcat_PRPS '], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['crew_category'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(skyc_nan,)","(crwcat_CPLT ,)","(crwcat_DSTU ,)","(crwcat_FENG ,)","(crwcat_FLTI ,)","(crwcat_KPLT ,)","(crwcat_OTHR ,)","(crwcat_PLT,)","(crwcat_PLT ,)","(crwcat_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,0,0,1,0


##### Crew Sex | `crew_sex`

In [160]:
ntsb08['crew_sex'].value_counts()

M    61561
F     2011
U      596
Name: crew_sex, dtype: int64

**Comment:** Include unknown with male category. 

In [161]:
#Replace U with M
ntsb08['crew_sex'] = np.where(ntsb08['crew_sex'] == 'U', 'M', ntsb08['crew_sex'])
#Replace M with 1
ntsb08['crew_sex'] = np.where(ntsb08['crew_sex'] == 'M', 1, 0)
#Verify
ntsb08['crew_sex'].value_counts()

1    62157
0     2023
Name: crew_sex, dtype: int64

##### Crew State | `crew_res_state`
Crew resident country was dropped in the *Cleaning and Initial Exploration* Jupyter Notebook (Notebook #1). Without city or country information, this feature is less impactful and will be dropped.

In [162]:
ntsb08 = ntsb08.drop(['crew_res_state'], axis=1)

#### Medical Certification | `med_certf`

In [163]:
ntsb08['med_certf'] = ntsb08['med_certf'].str.upper()
ntsb08['med_certf'].value_counts()

CL3     26874
CL2     22399
CL1     11191
NONE     1680
UNK      1512
CL3       466
SPRT       46
Name: med_certf, dtype: int64

**Comment:** This is feature requires some slight cleaning. Class 3 (CL 3) medical certificates are the dominant category value and will consolidated and the unknowns will also be included in CL3.

In [164]:
#Replace 'CL3 ' with 'CL3'
ntsb08['med_certf']= np.where(ntsb08['med_certf'] == 'CL3 ', 'CL3', ntsb08['med_certf'])
#Replace 'UNK' with 'CL3'
ntsb08['med_certf'] = np.where(ntsb08['med_certf'] == 'UNK ', 'CL3', ntsb08['med_certf'])
#Replace 'UNK' with 'CL3'
ntsb08['med_certf'] = np.where(ntsb08['med_certf'] == 'UNK', 'CL3', ntsb08['med_certf'])
#Verify
ntsb08['med_certf'].value_counts()

CL3     28852
CL2     22399
CL1     11191
NONE     1680
SPRT       46
Name: med_certf, dtype: int64

In [165]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['med_certf'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('medcrt_')

# Add original back in (just to check)
encoded['med_certf'] = ntsb08['med_certf']

# Show
encoded.head()

[array(['CL1 ', 'CL2 ', 'CL3', 'NONE', 'SPRT', nan], dtype=object)]


,medcrt_CL1,medcrt_CL2,medcrt_CL3,medcrt_NONE,medcrt_SPRT,medcrt_nan,med_certf
0,0,1,0,0,0,0,CL2
1,1,0,0,0,0,0,CL1
2,1,0,0,0,0,0,CL1
3,0,0,1,0,0,0,CL3
4,0,1,0,0,0,0,CL2


In [166]:
#Drop the final column
encoded= encoded.drop(['med_certf'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

medcrt_CL3     28852
medcrt_CL2     22399
medcrt_CL1     11191
medcrt_NONE     1680
medcrt_SPRT       46
medcrt_nan        12
dtype: int64

In [167]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['medcrt_SPRT'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['med_certf'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(crwcat_KPLT ,)","(crwcat_OTHR ,)","(crwcat_PLT,)","(crwcat_PLT ,)","(crwcat_nan,)","(medcrt_CL1 ,)","(medcrt_CL2 ,)","(medcrt_CL3,)","(medcrt_NONE,)","(medcrt_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,1,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,0,1,0,1,0,0,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,1,0,1,0,0,0,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,1,0,0,1,0,0,0


##### Valid Medical Certificate | `med_crtf_vldty`

In [168]:
ntsb08['med_crtf_vldty'] = ntsb08['med_crtf_vldty'].str.upper()
ntsb08['med_crtf_vldty'].value_counts()

VWW     32398
VNOW    26804
UNK      1770
NONE     1466
EXP      1427
NV        303
Name: med_crtf_vldty, dtype: int64

**Comment:** This feature will be condensed into 'valid' (1) or 'not valid'. 'VWW' and 'VNOW' indicate a valid medical certificate and unknowns will be included as valid medical certificates. All other categories will be considered 'not' valid

In [169]:
#Replace UNK with VWW
ntsb08['med_crtf_vldty'] = np.where(ntsb08['med_crtf_vldty'] == 'UNK', 'VWW', ntsb08['med_crtf_vldty'])
#Replace VNOW with VWW
ntsb08['med_crtf_vldty'] = np.where(ntsb08['med_crtf_vldty'] == 'VNOW', 'VWW', ntsb08['med_crtf_vldty'])
#Replace VWW with '1'
ntsb08['med_crtf_vldty'] = np.where(ntsb08['med_crtf_vldty'] == 'VWW', 1, 0)
#verify
ntsb08['med_crtf_vldty'].value_counts()

1    59202
0     4978
Name: med_crtf_vldty, dtype: int64

##### Seatbelts Used | `seatbelts_used`

In [170]:
ntsb08['seatbelts_used'] = ntsb08['seatbelts_used'].str.upper()
ntsb08['seatbelts_used'].value_counts()

Y    55048
U     8429
N      648
A       43
Name: seatbelts_used, dtype: int64

In [171]:
print('Percent Unknown: ', ntsb08['seatbelts_used'].str.count('U').sum()/ntsb08.shape[0]*100)

Percent Unknown:  13.133374883141165


**Comment:** The unknown rows comprise 13% and there is not a reasonable understanding to comfortably make a decision on how to impute them. This feature will be dropped. 

In [172]:
ntsb08 = ntsb08.drop(['seatbelts_used'], axis=1)

##### Crew Toxicology Test Performed | `crew_tox_perf`

In [173]:
ntsb08['crew_tox_perf'] = ntsb08['crew_tox_perf'].str.upper()
ntsb08['crew_tox_perf'].value_counts()

N    48861
Y     9954
U     5353
Name: crew_tox_perf, dtype: int64

**Comment:** The unknown rows will be included in the no column.

In [174]:
#Binarize column, yes imputed with '1'
ntsb08['crew_tox_perf'] = np.where(ntsb08['crew_tox_perf'] == 'Y', 1, 0)
ntsb08['crew_tox_perf'].value_counts()

0    54226
1     9954
Name: crew_tox_perf, dtype: int64

##### Pilot Seating Location | `seat_occ_pic`

In [175]:
ntsb08['seat_occ_pic'] = ntsb08['seat_occ_pic'].str.upper()
ntsb08['seat_occ_pic'].value_counts()

LEFT    37673
UNK      8926
RGT      6488
FRT      5445
CTR      3546
REAR     1778
SNGL      312
Name: seat_occ_pic, dtype: int64

**Comment:** This feature indicates where the pilot in command was sitting during the event. Sitting in the front ('FRT') seems reasonable descriptor for most cockpits and unknowns will be replaced with 'FRT'. 

In [176]:
#Replace 'UNK' with 'FRT'
ntsb08['seat_occ_pic'] = np.where(ntsb08['seat_occ_pic'] == 'UNK ', 'FRT', ntsb08['seat_occ_pic'])
ntsb08['seat_occ_pic'] = np.where(ntsb08['seat_occ_pic'] == 'UNK ', 'FRT', ntsb08['seat_occ_pic'])
#Verify
ntsb08['seat_occ_pic'].value_counts()

LEFT    37673
FRT      8926
RGT      6488
FRT      5445
CTR      3546
REAR     1778
SNGL      312
Name: seat_occ_pic, dtype: int64

In [177]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder()
subcategory = pd.DataFrame(ntsb08['seat_occ_pic'])

# Fit the OneHotEncoder to the subcategory column and transform into a DataFrame
encoded = ohe.fit_transform(subcategory)

# Convert from sparse matrix to dense
dense_array = encoded.toarray()

print(ohe.categories_)

# Put into a dataframe to get column names
encoded = pd.DataFrame(dense_array, columns=ohe.categories_, dtype=int)

#Add prefix to column names
encoded= encoded.add_prefix('pltseat_')

# Add original back in (just to check)
encoded['seat_occ_pic'] = ntsb08['seat_occ_pic']

# Show
encoded.head()

[array(['CTR ', 'FRT', 'FRT ', 'LEFT', 'REAR', 'RGT ', 'SNGL', nan],
      dtype=object)]


,pltseat_CTR,pltseat_FRT,pltseat_FRT,pltseat_LEFT,pltseat_REAR,pltseat_RGT,pltseat_SNGL,pltseat_nan,seat_occ_pic
0,0,0,0,1,0,0,0,0,LEFT
1,0,0,0,1,0,0,0,0,LEFT
2,0,0,0,1,0,0,0,0,LEFT
3,0,0,0,1,0,0,0,0,LEFT
4,0,0,0,1,0,0,0,0,LEFT


In [178]:
#Drop the final column
encoded= encoded.drop(['seat_occ_pic'], axis=1)

#View total count across columns
encoded.sum().sort_values(ascending=False)

pltseat_LEFT    37673
pltseat_FRT      8926
pltseat_RGT      6488
pltseat_FRT      5445
pltseat_CTR      3546
pltseat_REAR     1778
pltseat_SNGL      312
pltseat_nan        12
dtype: int64

In [179]:
#Drop 1x column to avoid collinearity
encoded = encoded.drop(['pltseat_SNGL'], axis=1)
#Concatenate back onto main dataframe
ntsb08 = pd.concat([ntsb08, encoded], axis=1)
#Drop original column
ntsb08 = ntsb08.drop(['seat_occ_pic'], axis=1)
#Check main dataframe
ntsb08.head()

,acft_missing,flight_plan_activated,cert_max_gr_wt,homebuilt,total_seats,num_eng,elt_install,elt_oper,elt_aided_loc_ev,certs_held,...,"(medcrt_CL3,)","(medcrt_NONE,)","(medcrt_nan,)","(pltseat_CTR ,)","(pltseat_FRT,)","(pltseat_FRT ,)","(pltseat_LEFT,)","(pltseat_REAR,)","(pltseat_RGT ,)","(pltseat_nan,)"
0,0.0,0.0,3800.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,1,0,0,0
1,0.0,0.0,750000.0,0.0,9.0,4.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,1,0,0,0
2,0.0,0.0,7369.0,0.0,6.0,2.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,1,0,0,0
3,0.0,0.0,2300.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,0
4,0.0,0.0,3800.0,0.0,6.0,1.0,1.0,1.0,0.0,1.0,...,0,0,0,0,0,0,1,0,0,0


---
<a id='Step-3'></a>
### 3. Confirm Dataset
---
Verify the data is all numerical and there are no more categorical features reamining. Double check for any nulls. In this case, nulls were generated during a cleaning step. There were only 12 rows, so those are dropped. Confirm the final shape of the dataset and review the dataset information prior to exporting.

**Note:** The export code cell is commented out. Remove the # and rerun the cell to export the CSV file used for modeling. 

In [180]:
#Review the categorical columns
print('Text Columns:\n') 
cat_list = list(ntsb08.select_dtypes(exclude='number'))
print(cat_list)
print('Remaining:', len(cat_list))

Text Columns:

[]
Remaining: 0


In [181]:
ntsb08.isna().sum().sum()

124416

In [182]:
ntsb08 = ntsb08.dropna()

In [183]:
ntsb08.shape

(64156, 10472)

In [184]:
ntsb08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64156 entries, 0 to 64167
Columns: 10472 entries, acft_missing to ('pltseat_nan',)
dtypes: float64(10368), int32(104)
memory usage: 5.0 GB


In [185]:
ntsb08.to_csv('data/ntsb08_model.csv')

---------
<a id='Step-4'></a>
### 4. Results & Summary
---------
This concludes the pre-processing of the NTSB dataset. Initially there were 64 features total of which 44 were categorical. After encdoing the dummy variables there are almost 10,500 features. When verifying the dataset was clean, there were 12 new rows containing which was a surprise. This could be investigated, however it is only 12 rows so the rows were dropped. 

Finally, collinearity between the independent should be checked. Collinearity or multicollinearity are indicative of correlation between two or more features. Linear regression, logistic regression and K-Nearest Neighbor models are all sensitive to multicollinearity and the independent features need to be independent of the dependent and independent variables. Highly correlated features or introducing multicollinearity can cause numeric issues, or the coefficients might be unreliable. Collinearity can be checked by examining correlation and using Variance Inflation Factor (VIF). Due to the size and number of features of this dataset, the laptop was unable to handle running collinearity tests creating a bias in the subsequent models. 

---
#### <div align = "right">Up Next:</div>
<div align = "right">Aviation Accident Captson Part III: Logistic Regression Model</div>